## pair counting - merge
<input type="checkbox" unchecked> 统计字节对频率(Pair counting) 在"词形片段"内部把相邻字节两两匹配、计算出现频率

<input type="checkbox" unchecked> 合并(Merge) 选取最高频(并列选最高字典序) pair 合并成新 token 重复到 vocab 满

### 问题抽象
现有 $n$ 组( $n = 2 \times 10^6 $ , 每组长度约 $120$ ~ $150$ )变长字符串 $str_i$ ($len(str_i) \leq 20,~i \in [0, \, n)$)，总长度 $\sum_{i = 0}^n len(str_i) \leq 150 \times 10 \times 2 \times 10^6 = 3 \times 10^9$，

$vocab$ 为 $token ~ id$ 对 $Unicode ~ byte(s)$ 的映射集合（也就是 $token$ 集合），初始化为 $[0, 256)$ 对应其十六进制数所代表的 $Unicode ~ byte$ (可以理解为 $ASCII ~ plus$) 

**持续执行如下操作：**

对每个字符串 ***所有最长 $token$*** 进行两两结合统计频率：

<aside>

初始状态每个字符是一个 $token$

`word` → `wo` : 1  `or` : 1  `rd` : 1

> 什么是***最长 $token$*** ？如 `newest` ，`est` 已分配 $token ~ id$，我们仅对 `ne` `ew` `west` 处理
> 
</aside>

将频率最高的一个 $pair$ （若并列则取最高字典序）分配新的 $token ~ id$ 

比如上述 `word` 则新分配 `279` → `wo`

**截止状态：**

最终整个序列已经分配了 $token ~ id$（几乎不可能） 或 $token ~ id$ 的数目 = $vocab\_size$

$token ~ id \leq vocab\_size = 10^4$

---

#### 输入

二维字符串数组

#### 输出

$vocab$ : 从 $token ~ id$ 到 $bytes$ 的映射

$merges$ : 产生的 合并 $pair$（按创建顺序排列）

In [1]:
import os, regex, time, json
from typing import List, Dict, Tuple
from collections import Counter, defaultdict
from multiprocessing import Pool, cpu_count

In [2]:
def _tokenize_batch_worker(args):
    """全局worker函数：分词一批文本"""
    batch_lines, PAT = args
    PAT_c = regex.compile(PAT)
    stories = []
    for line in batch_lines:
        tokens = [m.group(0).encode("utf-8") for m in PAT_c.finditer(line)]
        if tokens:
            stories.append(tokens)
    return stories

In [3]:
def _count_pairs_batch_worker(args):
    """全局worker函数：统计一批stories的pairs"""
    story_batch, special_set = args
    pairs = Counter()
    for story in story_batch:
        for i in range(len(story) - 1):
            if story[i] not in special_set and story[i+1] not in special_set:
                pairs[(story[i], story[i+1])] += 1
    return pairs

In [4]:
def _apply_merge_batch_worker(args):
    """全局worker函数：并行执行merge"""
    story_batch, a, b, ab, special_set = args
    local_old_deltas = Counter()
    local_new_deltas = Counter()
    
    for story in story_batch:
        i = 0
        while i < len(story) - 1:
            if story[i] == a and story[i+1] == b:
                # 记录旧pairs
                if i > 0 and story[i-1] not in special_set:
                    local_old_deltas[(story[i-1], a)] += 1
                if i + 2 < len(story) and story[i+2] not in special_set:
                    local_old_deltas[(b, story[i+2])] += 1
                
                # merge
                story[i] = ab
                story.pop(i+1)
                
                # 记录新pairs
                if i > 0 and story[i-1] not in special_set:
                    local_new_deltas[(story[i-1], ab)] += 1
                if i + 1 < len(story) and story[i+1] not in special_set:
                    local_new_deltas[(ab, story[i+1])] += 1
                
                i += 1
            else:
                i += 1
    
    return story_batch, local_old_deltas, local_new_deltas

In [5]:
def train_bpe(
    input_path: str,
    vocab_size: int,
    special_tokens: List[str],
    PAT: str,
    output_dir: str = "./bpe_model",
    accurate_ratio: float = 0.6,
    num_proc: int = None,
    recount_interval: int = 100,
    progress_interval: int = 100  # 每N次merge打印一次进度
) -> Tuple[Dict[int, bytes], List[Tuple[bytes, bytes]]]:
    """
    清爽版BPE训练 - 避免进度条冲突
    
    优化：
    1. 使用简单打印代替tqdm（避免多进程冲突）
    2. 并行化merge操作
    3. 清晰的阶段显示
    
    资源预估（2.2M stories）：
    - 内存: 15-25GB
    - 时间: 20-40分钟
    - 质量: 95%+
    """
    
    start_time = time.time()
    
    if num_proc is None:
        num_proc = max(1, cpu_count() - 2)
    
    print(f"\n{'='*80}")
    print(f"🚀 Optimized Hybrid BPE Training")
    print(f"{'='*80}")
    print(f"  Accurate updates: {accurate_ratio*100:.0f}% of merges")
    print(f"  Workers: {num_proc}")
    print(f"  Recount interval: {recount_interval}")
    print(f"{'='*80}\n")
    
    # --------------------------
    # 1️⃣ 初始化 vocab
    # --------------------------
    vocab: Dict[int, bytes] = {i: bytes([i]) for i in range(256)}
    idx = 256
    for tok in special_tokens:
        vocab[idx] = tok.encode("utf-8")
        idx += 1
    
    special_set = set(t.encode("utf-8") for t in special_tokens)
    
    # --------------------------
    # 2️⃣ 并行加载和分词
    # --------------------------
    print("[Step 1/3] Loading and tokenizing...")
    step_start = time.time()
    
    with open(input_path, "rb") as f:
        text = f.read().decode("utf-8", errors="ignore")
    
    for tok in special_tokens:
        text = text.replace(tok, f" {tok} ")
    
    lines = [line for line in text.split("\n") if line.strip()]
    
    # 方案1：向上取整，确保最多num_proc个batches
    batch_size = (len(lines) + num_proc - 1) // num_proc  # 等价于 math.ceil(len(lines) / num_proc)
    batches = [lines[i:i+batch_size] for i in range(0, len(lines), batch_size)]
    batch_args = [(batch, PAT) for batch in batches]
    
    print(f"  Tokenizing {len(lines):,} lines with {len(batches)} workers (≤{num_proc})...")
    
    with Pool(num_proc) as pool:
        story_batches = pool.map(_tokenize_batch_worker, batch_args)
    
    all_stories = [story for batch in story_batches for story in batch]
    total_tokens = sum(len(story) for story in all_stories)
    
    step_time = time.time() - step_start
    print(f"  ✓ Loaded {len(all_stories):,} stories, {total_tokens:,} tokens")
    print(f"  ✓ Time: {step_time:.1f}s\n")
    
    # --------------------------
    # 3️⃣ 并行统计初始pairs
    # --------------------------
    print("[Step 2/3] Counting initial pairs...")
    step_start = time.time()
    
    stories_per_worker = max(1, len(all_stories) // num_proc)
    story_batches = [
        all_stories[i:i+stories_per_worker] 
        for i in range(0, len(all_stories), stories_per_worker)
    ]
    count_args = [(batch, special_set) for batch in story_batches]
    
    with Pool(num_proc) as pool:
        partial_counts = pool.map(_count_pairs_batch_worker, count_args)
    
    pair_counts = Counter()
    for pc in partial_counts:
        pair_counts.update(pc)
    
    step_time = time.time() - step_start
    print(f"  ✓ Found {len(pair_counts):,} unique pairs")
    print(f"  ✓ Time: {step_time:.1f}s\n")
    
    # --------------------------
    # 4️⃣ 混合策略merge（并行化）
    # --------------------------
    merges: List[Tuple[bytes, bytes]] = []
    num_merges = vocab_size - len(vocab)
    accurate_merges = int(num_merges * accurate_ratio)
    
    print(f"[Step 3/3] Performing {num_merges:,} merges")
    print(f"  Phase 1: {accurate_merges:,} accurate merges (parallel)")
    print(f"  Phase 2: {num_merges - accurate_merges:,} approximate merges")
    print(f"  Progress will be shown every {progress_interval} merges\n")
    
    phase_start = time.time()
    
    # 预先分配story batches
    stories_per_worker = max(1, len(all_stories) // num_proc)
    story_batches_fixed = [
        all_stories[i:i+stories_per_worker] 
        for i in range(0, len(all_stories), stories_per_worker)
    ]
    
    for merge_iter in range(num_merges):
        if not pair_counts:
            print(f"  ⚠ No more pairs to merge at iteration {merge_iter}")
            break
        
        merge_start = time.time()  # 计时单次merge
        
        best_pair = max(pair_counts, key=pair_counts.get)
        best_count = pair_counts[best_pair]  # 记录频率
        a, b = best_pair
        ab = a + b
        
        vocab[idx] = ab
        merges.append(best_pair)
        idx += 1
        
        use_accurate = merge_iter < accurate_merges
        
        if use_accurate:
            # ===== 并行准确更新 =====
            merge_args = [(batch, a, b, ab, special_set) for batch in story_batches_fixed]
            
            with Pool(num_proc) as pool:
                results = pool.map(_apply_merge_batch_worker, merge_args)
            
            # 更新story batches
            for i, (updated_batch, _, _) in enumerate(results):
                story_batches_fixed[i] = updated_batch
            
            all_stories = [story for batch in story_batches_fixed for story in batch]
            
            # 合并deltas
            total_old_deltas = Counter()
            total_new_deltas = Counter()
            for _, old_deltas, new_deltas in results:
                total_old_deltas.update(old_deltas)
                total_new_deltas.update(new_deltas)
            
            # 更新pair_counts
            pair_counts[best_pair] = 0
            
            for pair, delta in total_old_deltas.items():
                pair_counts[pair] -= delta
                if pair_counts[pair] <= 0:
                    del pair_counts[pair]
            
            for pair, delta in total_new_deltas.items():
                pair_counts[pair] += delta
        
        else:
            # ===== 近似更新 =====
            pair_counts.pop(best_pair, None)
            
            if (merge_iter - accurate_merges) % recount_interval == 0:
                recount_start = time.time()
                
                count_args = [(batch, special_set) for batch in story_batches_fixed]
                with Pool(num_proc) as pool:
                    partial_counts = pool.map(_count_pairs_batch_worker, count_args)
                
                pair_counts = Counter()
                for pc in partial_counts:
                    pair_counts.update(pc)
                
                recount_time = time.time() - recount_start
                print(f"  [Recount at merge {merge_iter}] Time: {recount_time:.1f}s")
        
        merge_time = time.time() - merge_start
        
        # 定期显示进度 + 实时性能监控
        if (merge_iter + 1) % progress_interval == 0 or merge_iter < 10:  # 前10次必显示
            elapsed = time.time() - phase_start
            rate = (merge_iter + 1) / elapsed
            remaining = (num_merges - merge_iter - 1) / rate if rate > 0 else 0
            percentage = (merge_iter + 1) / num_merges * 100
            
            phase_name = "Accurate" if use_accurate else "Approximate"
            print(f"  [{phase_name}] {merge_iter+1}/{num_merges} ({percentage:.1f}%) | "
                  f"Last merge: {merge_time:.1f}s (freq={best_count:,}) | "
                  f"Avg rate: {rate:.2f}/s | ETA: {remaining/60:.1f}min")
        
        # 阶段切换提示
        if merge_iter == accurate_merges - 1:
            phase_time = time.time() - phase_start
            print(f"\n  ✓ Accurate phase complete ({phase_time/60:.1f}min)")
            print(f"  → Switching to approximate mode (faster)...\n")
            phase_start = time.time()
    
    total_merge_time = time.time() - (step_start - step_time)
    print(f"\n  ✓ All merges complete")
    print(f"  ✓ Time: {total_merge_time/60:.1f}min\n")
    
    # --------------------------
    # 5️⃣ 保存模型
    # --------------------------
    print("[Step 4/4] Saving model...")
    os.makedirs(output_dir, exist_ok=True)
    vocab_path = os.path.join(output_dir, "vocab.json")
    merges_path = os.path.join(output_dir, "merges.txt")
    
    vocab_serialized = {str(k): v.decode("utf-8", errors="replace") for k, v in vocab.items()}
    with open(vocab_path, "w", encoding="utf-8") as f:
        json.dump(vocab_serialized, f, ensure_ascii=False, indent=2)
    
    with open(merges_path, "w", encoding="utf-8") as f:
        for a, b in merges:
            f.write(f"{a.decode('utf-8', errors='replace')} {b.decode('utf-8', errors='replace')}\n")
    
    elapsed = time.time() - start_time
    print(f"  ✓ Saved to {output_dir}\n")
    
    print(f"{'='*80}")
    print(f"✅ BPE training completed!")
    print(f"{'='*80}")
    print(f"  Total time: {elapsed/60:.1f} minutes ({elapsed:.0f}s)")
    print(f"  Final vocab size: {len(vocab)}")
    print(f"  Total merges: {len(merges)}")
    print(f"  Average rate: {len(merges)/elapsed:.2f} merges/s")
    print(f"  Output directory: {output_dir}")
    print(f"{'='*80}\n")
    
    return vocab, merges

In [6]:
train_path = "/home/winbeau/Study/1-transformer/datasets/TinyStories/txt/valid_with_eot.txt"
PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
vocab, merges = train_bpe(
    input_path=train_path,
    vocab_size=10000,
    special_tokens=["<|endoftext|>"],
    PAT=PAT,
    output_dir="./bpe_model_hybrid",
    accurate_ratio=0.6,  # 前60%精确，后40%近似
    num_proc=12, 
    recount_interval=100, # 控制最后近似更新的重新统计更新频率 | 默认100次
    progress_interval=100  # 每100次merge显示一次 | 前10次必显示
)


🚀 Optimized Hybrid BPE Training
  Accurate updates: 60% of merges
  Workers: 12
  Recount interval: 100

[Step 1/3] Loading and tokenizing...
  Tokenizing 21,990 lines with 12 workers (≤12)...
  ✓ Loaded 21,990 stories, 4,620,113 tokens
  ✓ Time: -6.4s

[Step 2/3] Counting initial pairs...
  ✓ Found 298,823 unique pairs
  ✓ Time: 2.2s

[Step 3/3] Performing 9,743 merges
  Phase 1: 5,845 accurate merges (parallel)
  Phase 2: 3,898 approximate merges
  Progress will be shown every 100 merges

  [Accurate] 1/9743 (0.0%) | Last merge: 9.1s (freq=52,107) | Avg rate: 0.11/s | ETA: 1476.8min
  [Accurate] 2/9743 (0.0%) | Last merge: -5.8s (freq=42,231) | Avg rate: 0.61/s | ETA: 267.9min
  [Accurate] 3/9743 (0.0%) | Last merge: 1.9s (freq=40,692) | Avg rate: 0.57/s | ETA: 282.5min
  [Accurate] 4/9743 (0.0%) | Last merge: 1.8s (freq=35,270) | Avg rate: 0.57/s | ETA: 283.5min
  [Accurate] 5/9743 (0.1%) | Last merge: 1.5s (freq=22,576) | Avg rate: 0.59/s | ETA: 277.0min
  [Accurate] 6/9743 (0.1%)